# Neural Network from Scratch in TensorFlow

# Task 1: Introduction and Importing Libraries

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nn_utils
%matplotlib inline

print('TensorFlow Version:', tf.__version__)

TensorFlow Version: 2.12.0


# Task 2: Initializing Neural Network

In [4]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.L = len(layers)
        self.num_features = layers[0] ##Number of features
        self.num_classes = layers[-1] ##Number of otput classes
        
        self.W = {}
        self.b = {}
        
        self.dW = {}
        self.db = {}
        
        self.setup()
        
    def setup(self):
        # Your code here
        for i in range(1,self.L):
          self.W[i] = tf.Variable(tf.random.normal(shape=(self.layers[i],self.layers[i-1])))
          self.b[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],1)))

# Task 3: Forward Pass

In [6]:
class NeuralNetwork(NeuralNetwork):
    def forward_pass(self, X): # X is input here
        A = tf.convert_to_tensor(X,dtype=tf.float32)
        for i in range(1,self.L):
          Z = tf.matmul(A,tf.transpose(self.W[i]))+tf.transpose(self.b[i])# Z is the linear output
          if i != self.L-1: # If the layers is not the final layer
            # Setting up activation for hidden layers
            A = tf.nn.relu(Z)
          else: 
            # Setting up AF for final layer
            A = Z
        return A

# Task 4: Computing Loss and Updating Parameters

In [ ]:
class NeuralNetwork(NeuralNetwork):
    # Your code here

# Task 5: Predict and Info Functions

In [ ]:
class NeuralNetwork(NeuralNetwork):
    def predict(self, X):
        # Your code here
    
    def info(self):
        num_params = 0
        for i in range(1, self.L):
            num_params += self.W[i].shape[0] * self.W[i].shape[1]
            num_params += self.b[i].shape[0]
        print('Input Features:', self.num_features)
        print('Number of Classes:', self.num_classes)
        print('Hidden Layers:')
        print('--------------')
        for i in range(1, self.L-1):
            print('Layer {}, Units {}'.format(i, self.layers[i]))
        print('--------------')
        print('Number of parameters:', num_params)

# Task 6: Training on Batch

In [ ]:
class NeuralNetwork(NeuralNetwork):
    def train_on_batch(self, X, Y, lr):
        # Your code here

# Task 7: Training on Complete Set

In [ ]:
class NeuralNetwork(NeuralNetwork):
    def train(self, x_train, y_train, x_test, y_test, epochs, steps_per_epoch, batch_size, lr):
        # Your code here

# Task 8: Application

# Task 9: Results